In [5]:
import requests

def fetch_labels(entity_ids):
    """
    Fetch labels for a given list of Wikidata entity IDs.

    Parameters:
    - entity_ids: List of Wikidata entity IDs.

    Returns:
    - A dictionary mapping each Wikidata entity ID to its label.
    """
    labels = {}
    # Wikidata API may have limits on the number of IDs per request; adjust if necessary.
    for i in range(0, len(entity_ids), 50):
        batch_ids = entity_ids[i:i+50]
        params = {
            'action': 'wbgetentities',
            'ids': '|'.join(batch_ids),
            'props': 'labels',
            'languages': 'en',
            'format': 'json'
        }
        response = requests.get("https://www.wikidata.org/w/api.php", params=params)
        response_json = response.json()

        for entity_id, entity in response_json['entities'].items():
            label = entity.get('labels', {}).get('en', {}).get('value', entity_id)
            labels[entity_id] = label
    return labels

def format_value(value, labels):
    """
    Convert a Wikidata value to a string representation, using labels for entity IDs.
    """
    if isinstance(value, dict):
        # For entity IDs, replace with label if available
        if 'id' in value:
            return labels.get(value['id'], value['id'])
        elif 'time' in value:
            return value['time']
        elif 'amount' in value:
            return value['amount']
        else:
            return str(value)
    else:
        return str(value)

def get_all_properties_with_labels(wikidata_id):
    """
    Fetch all properties and their values for a given Wikidata ID, 
    including labels for properties and their entity values.

    Parameters:
    - wikidata_id: The Wikidata ID of the entity.

    Returns:
    - A dictionary with property labels as keys and lists of their labeled values as values.
    """
    # Initial API call to get all claims/properties for the entity
    params = {
        'action': 'wbgetentities',
        'ids': wikidata_id,
        'props': 'claims',
        'format': 'json'
    }
    response = requests.get("https://www.wikidata.org/w/api.php", params=params)
    claims = response.json()['entities'][wikidata_id]['claims']

    # Collect property IDs and value entity IDs for label fetching
    prop_ids = list(claims.keys())
    value_entity_ids = set()
    for prop_id in prop_ids:
        for claim in claims[prop_id]:
            if 'datavalue' in claim['mainsnak']:
                data_value = claim['mainsnak']['datavalue'].get('value')
                if isinstance(data_value, dict) and 'id' in data_value:
                    value_entity_ids.add(data_value['id'])

    # Fetch labels for all property IDs and value entity IDs
    all_labels = fetch_labels(list(set(prop_ids) | value_entity_ids))

    # Construct the result dictionary with labels
    result = {}
    for prop_id in prop_ids:
        prop_label = all_labels.get(prop_id, prop_id)
        result[prop_label] = []
        for claim in claims[prop_id]:
            if 'datavalue' in claim['mainsnak']:
                data_value = claim['mainsnak']['datavalue'].get('value')
                formatted_value = format_value(data_value, all_labels)
                result[prop_label].append(formatted_value)

    return result

# Example usage
wikidata_id = 'Q155979'
properties_with_labels = get_all_properties_with_labels(wikidata_id)

for prop, values in properties_with_labels.items():
    print(f"{prop}: {', '.join(values)}")


GND ID: 1024799212
member of: Russian Opposition Coordination Council, Yale World Fellows
sex or gender: male
Library of Congress authority ID: n2012030899
VIAF ID: 254613339, 304693850
educated at: Finance University under the Government of the Russian Federation, Yale University, Peoples' Friendship University of Russia, Yale World Fellows
image: Alexey Navalny (cropped) 2.jpg
member of political party: Yabloko, Progress Party, Russia of the Future
Commons category: Alexey Navalny
Freebase ID: /m/0dlllc6
employer: Anti-Corruption Foundation, Aeroflot
date of birth: +1976-06-04T00:00:00Z
religion or worldview: Eastern Orthodoxy
native language: Russian
country of citizenship: Soviet Union, Russia
field of work: politics, jurisprudence
place of birth: Butyn
instance of: human
IdRef ID: 161623972
described by source: Lentapedia, Navalny
official website URL: https://navalny.com
given name: Alexey
significant event: Yves Rocher case, poisoning of Alexei Navalny, Kirovles trial, incarcera

In [6]:
import requests

def fetch_wikipedia_page_title(wikidata_id):
    """
    Fetch the Wikipedia page title for a given Wikidata ID.

    Parameters:
    - wikidata_id: The Wikidata ID of the entity.

    Returns:
    - The Wikipedia page title as a string.
    """
    params = {
        'action': 'wbgetentities',
        'ids': wikidata_id,
        'props': 'sitelinks',
        'sitefilter': 'enwiki',
        'format': 'json'
    }
    response = requests.get("https://www.wikidata.org/w/api.php", params=params)
    response_json = response.json()
    sitelinks = response_json['entities'][wikidata_id].get('sitelinks', {})
    enwiki = sitelinks.get('enwiki', {}).get('title', '')
    return enwiki

def fetch_wikipedia_content(page_title):
    """
    Fetch the content of a Wikipedia page given its title.

    Parameters:
    - page_title: The title of the Wikipedia page.

    Returns:
    - The content of the Wikipedia page as a string.
    """
    params = {
        'action': 'query',
        'format': 'json',
        'titles': page_title,
        'prop': 'extracts',
        'exintro': True,  # Get the introduction section of the page
        'explaintext': True,  # Return plain text content
    }
    response = requests.get("https://en.wikipedia.org/w/api.php", params=params)
    response_json = response.json()
    page = next(iter(response_json['query']['pages'].values()))
    content = page.get('extract', '')
    return content

# Example usage
wikidata_id = 'Q155979'  # Example: Douglas Adams
page_title = fetch_wikipedia_page_title(wikidata_id)
wikipedia_content = fetch_wikipedia_content(page_title)

print(f"Wikipedia Intro for '{page_title}':\n{wikipedia_content}")


Wikipedia Intro for 'Alexei Navalny':
Alexei Anatolyevich Navalny (Russian: Алексей Анатольевич Навальный, IPA: [ɐlʲɪkˈsʲej ɐnɐˈtolʲjɪvʲɪtɕ nɐˈvalʲnɨj]; 4 June 1976 – 16 February 2024) was a Russian opposition leader, lawyer, anti-corruption activist, and political prisoner. He organised anti-government demonstrations and ran for office to advocate reforms against corruption in Russia and against President Vladimir Putin and his government. Navalny was a Russian Opposition Coordination Council member, the leader of the Russia of the Future party and founder of the Anti-Corruption Foundation (FBK). He was recognised by Amnesty International as a prisoner of conscience, and was awarded the Sakharov Prize for his work on human rights.As of 2021, Navalny had more than six million YouTube subscribers; through his social media channels, he and his team published material about corruption in Russia, organised political demonstrations and promoted his campaigns. In a 2011 radio interview, he d

# Find ALL pages relating to the entity

In [8]:
import requests

def get_wikipedia_title_from_wikidata_id(wikidata_id):
    """Get the English Wikipedia page title for a given Wikidata ID."""
    wikidata_url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbgetentities',
        'ids': wikidata_id,
        'format': 'json',
        'props': 'sitelinks',
        'sitefilter': 'enwiki'
    }
    response = requests.get(wikidata_url, params=params)
    data = response.json()
    title = data['entities'][wikidata_id]['sitelinks']['enwiki']['title']
    return title

def find_links_to_wikipedia_page(page_title):
    """Find all English Wikipedia pages that link to a specific page."""
    wikipedia_url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'list': 'backlinks',
        'bltitle': page_title,
        'bllimit': 'max',
        'format': 'json'
    }
    links = []
    while True:
        response = requests.get(wikipedia_url, params=params)
        data = response.json()
        backlinks = data['query']['backlinks']
        links.extend([link['title'] for link in backlinks])
        
        if 'continue' in data:
            params['blcontinue'] = data['continue']['blcontinue']
        else:
            break
    
    return links

# Example usage
wikidata_id = 'Q42'  # Example: Douglas Adams
page_title = get_wikipedia_title_from_wikidata_id(wikidata_id)
links = find_links_to_wikipedia_page(page_title)

print(f"Pages linking to '{page_title}':")
for link in links:
    print(link)


Pages linking to 'Douglas Adams':
The Ashes
Ben Bova
List of comedians
Cetacea
Detective fiction
Doctor Who
List of English-language poets
George R. R. Martin
Graham Chapman
Gin and tonic
Harlan Ellison
HyperCard
List of humorists
Infocom
Interactive fiction
Jabberwocky
Liar paradox
Matt Groening
Monty Python
May 25
May 11
March 11
Meaning of life
Poul Anderson
Pun
Q.E.D.
Richard Dawkins
Talk:Rape/Archive 5
List of fictional robots and androids
Steven Spielberg
Slartibartfast
Samuel Taylor Coleridge
Steve Ditko
Trilogy
The Hitchhiker's Guide to the Galaxy
World Wide Web
Westwood Studios
Zork
Z-machine
1970s
1952
1978
1979
Cambridge
Dorset
Activision
Pseudonym
Greg Bear
From Hell
Robert Asprin
St John's College, Cambridge
Suspended (video game)
Eleanor Bron
Dirk Gently's Holistic Detective Agency
Terry Jones
Alice's Adventures in Wonderland
Marion Zimmer Bradley
List of science-fiction authors
Betelgeuse
Thursday
Innsbruck
The Salmon of Doubt
Foundation series
Mostly Harmless
Stephen Fr